# Import librerias

In [1]:
import requests
import json
import pandas as pd
from collections import Counter
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from datetime import datetime


In [2]:
print("Trabajaremos con las siguientes versiones\n")
print("Pandas",pd.__version__)
print("Numpy",np.__version__)
print("Seaborn", sns.__version__)
!python --version

Trabajaremos con las siguientes versiones

Pandas 2.0.2
Numpy 1.25.0
Seaborn 0.12.2
Python 3.11.3


# Funciones

In [3]:
def import_data(resource_id):

    #Define base url and parameters
    url = 'https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search'
    params = {'resource_id': resource_id, 'limit': 100000000000000} # Change limit size of the retrieved rows: https://docs.ckan.org/en/latest/maintaining/datastore.html#the-data-api  (see parameters in the datastore_search request)

    #Conect to API
    response_API = requests.get(url, params=params)
    if response_API.status_code != 200:
        raise Exception('Could not get data from the API')

    #Get data from API
    data = response_API.json()

    #Get data records
    data_records = data['result']['records']

    #Convert json into a dataframe
    df = pd.json_normalize(data_records)

    return df

In [4]:
# Creamos una función simple para hacer un report simple sobre los DataFrame suministrados
def report_df(dataframe, verbose = True):
    print("El tamaño del dataset es", dataframe.shape[0], "filas y", dataframe.shape[1], "columnas")
    print()
    print(dataframe.info(verbose = verbose))
    total_nulos = dataframe.isnull().sum().sum()
    print()
    print(f"Tenemos un total de {total_nulos} nulos")
    print()
    print("Hay", dataframe.duplicated().sum(), "duplicados")


# Load data

In [5]:
df_cens = import_data('c933fa10-a965-4509-8de6-e519281dc245').drop(columns = '_id')
df_cens.head()


,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
0,"4584527,7",Orange,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,movi,4584527,3644423DF3834D,10,ALTRES,ALTRES,SERVEIS,89,433707,31/03/2014,2012-05-16,Sant Martí,"433707,7",serv
1,"4584524,4",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584524,3644423DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,89,433701,31/03/2014,2012-05-16,Sant Martí,"433701,1",locb
2,4584520,La Llave de oro,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,inmo,4584520,3644426DF3834D,10,ACTIVITATS IMMOBILIÀRIES,SERVEIS,SERVEIS,91,433678,31/03/2014,2012-05-16,Sant Martí,"433678,8",serv
3,"4584515,5",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584515,3644426DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,91,433664,31/03/2014,2012-05-16,Sant Martí,"433664,4",locb
4,"4584511,1",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584511,3644426DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,93,433649,31/03/2014,2012-05-16,Sant Martí,"433649,9",locb


# EDA

In [6]:
## Observamos los datos que tenemos 
report_df(df_cens)

df_cens.head(2)

El tamaño del dataset es 67433 filas y 19 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67433 entries, 0 to 67432
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Y_ED50         67433 non-null  object
 1   NOM            67433 non-null  object
 2   CBARRI         67433 non-null  object
 3   NBARRI         67433 non-null  object
 4   CARRCADAST     67433 non-null  object
 5   CODI_EIXOS     67433 non-null  object
 6   Y              67433 non-null  object
 7   RCNUMBER       67433 non-null  object
 8   CDISTRI        67433 non-null  object
 9   BCN3           67433 non-null  object
 10  BCN2           67433 non-null  object
 11  BCN1           67433 non-null  object
 12  DOORNUM        67433 non-null  object
 13  X              67433 non-null  object
 14  ULTIMA_VISITA  67433 non-null  object
 15  DATAINICI      67433 non-null  object
 16  NDISTRIC       67433 non-null  object
 17  X_ED50         67

,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
0,"4584527,7",Orange,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,movi,4584527,3644423DF3834D,10,ALTRES,ALTRES,SERVEIS,89,433707,31/03/2014,2012-05-16,Sant Martí,"433707,7",serv
1,"4584524,4",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584524,3644423DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,89,433701,31/03/2014,2012-05-16,Sant Martí,"433701,1",locb


In [7]:
# Examinamos los duplicados existentes
df_cens[df_cens.duplicated()]

,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
125,"4583986,6",Local Buit,06,la Sagrada Família,AV DIAGONAL,localb,4583986,1740810DF3814B,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,298,431770,31/03/2014,2013-12-09,Eixample,431770,locb
150,"4583601,1",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,4583601,0835306DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,319,430826,31/03/2014,2013-12-31,Eixample,"430826,6",locb
179,4583750,Local Buit,06,la Sagrada Família,AV DIAGONAL,localb,4583750.37,1037609DF3813G,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,336,431055.96,31/03/2014,2011-07-06,Eixample,"431055,5",locb
208,"4583706,654",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,45837065477,0837728DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,358,43086864201411,31/03/2014,2013-12-31,Eixample,"430868,864",locb
266,"4583499,9",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,4583499.89,0235724DF3803E,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,414,430274.69,31/03/2014,2011-07-06,Eixample,"430274,4",locb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67374,"4588284,4",Varietats,48,la Guineueta,RB CAÃADOR,variet,4588284,0682301DF3808D,08,ALTRES,ALTRES,ALTRES,1,430536,09/03/2014,2014-03-09,Nou Barris,"430536,6",ALTRES
67387,"4582415,511",LLI privada Ninets,08,l Antiga Esquerra de l Eixample,ValÞncia,educ,45824151104439,,02,ENSENYAMENT,SERVEIS,SERVEIS,218-220,42971322259258,16/04/2013,2013-04-16,Eixample,"429713,322",other
67393,"4587906,6",Varietats,48,la Guineueta,RB FABRA I PUIG,variet,4587906,0479903DF3807G,08,ALTRES,ALTRES,ALTRES,474,430454,03/03/2014,2014-03-09,Nou Barris,"430454,4",ALTRES
67400,4587970,Varietats,48,la Guineueta,RB FABRA I PUIG,variet,4587970,0479902DF3807G,08,ALTRES,ALTRES,ALTRES,478,430463,03/03/2014,2014-03-09,Nou Barris,"430463,3",ALTRES


In [8]:
# Comprobamos si estos duplicados, realmente lo son
df_cens[df_cens['RCNUMBER'] == '0837728DF3803F']

,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
207,"4583706,654",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,45837065477,0837728DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,358,43086864201411,31/03/2014,2013-12-31,Eixample,"430868,864",locb
208,"4583706,654",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,45837065477,0837728DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,358,43086864201411,31/03/2014,2013-12-31,Eixample,"430868,864",locb


In [9]:
# Decidimos eliminar duplicados, viendo que tiene sentido
df_cens.drop_duplicates(inplace=True)

In [10]:
# Eliminamos columnas que no hacen falta
df_cens =  df_cens.drop(['Y_ED50', 'X_ED50', 'X', 'Y', 
                         'CARRCADAST', 'DOORNUM',
                         'ULTIMA_VISITA', 'DATAINICI',
                         'RCNUMBER'], axis=1)

In [11]:
# Comprobamos los datos que contenemos 
df_cens.describe(exclude=np.number).T

,count,unique,top,freq
NOM,66444,45556,Local Buit,9228
CBARRI,66444,74,07,4142
NBARRI,66444,74,la Dreta de l Eixample,4142
CODI_EIXOS,66444,120,localb,9715
CDISTRI,66444,11,02,14931
BCN3,66444,41,ALTRES,17930
BCN2,66444,9,SERVEIS,30785
BCN1,66444,4,SERVEIS,31244
NDISTRIC,66444,11,Eixample,14931
M_CODI_EIXOS,66444,11,serv,16895


In [12]:
# Probamos los valores que contiene esta columna para entender mejor nuestro dataset
df_cens['NBARRI'].value_counts()

NBARRI
la Dreta de l Eixample             4142
la Vila de Gràcia                  3481
Sant Gervasi - Galvany             2947
l Antiga Esquerra de l Eixample    2810
el Raval                           2540
                                   ... 
#N/A                                 21
Baró de Viver                        21
Torre Baró                           20
Vallbona                              8
la Clota                              6
Name: count, Length: 74, dtype: int64

In [13]:
# Remplazamos los valores nulos para que se puedan detectar
df_cens.replace('#N/A',np.nan,inplace=True)

In [14]:
# Comprobamos que se hizo el cambio correctamente
df_cens[df_cens.isnull().any(axis=1)].head(2)

,NOM,CBARRI,NBARRI,CODI_EIXOS,CDISTRI,BCN3,BCN2,BCN1,NDISTRIC,M_CODI_EIXOS
3556,CLUB DE PETANCA CASA OLIVA,NaN,NaN,espo,NaN,ALTRES,ALTRES,ALTRES,NaN,ALTRES
14999,Institut Lluís Vives,NaN,NaN,educ,NaN,ENSENYAMENT,SERVEIS,SERVEIS,NaN,other


In [15]:
# Eliminamos valores nulos
df_cens.dropna(inplace=True)
df_cens.reset_index(drop=True,inplace=True)

In [16]:
# Renombramos columnas para trabajar más fácilmente
df_cens.rename(columns={'CBARRI': 'Codi_Barri', 'NBARRI': 'Nom_Barri', 'CDISTRI': 'Codi_Districte', 'NDISTRIC': 'Nom_Districte'}, inplace=True)

In [17]:
df_cens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66423 entries, 0 to 66422
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NOM             66423 non-null  object
 1   Codi_Barri      66423 non-null  object
 2   Nom_Barri       66423 non-null  object
 3   CODI_EIXOS      66423 non-null  object
 4   Codi_Districte  66423 non-null  object
 5   BCN3            66423 non-null  object
 6   BCN2            66423 non-null  object
 7   BCN1            66423 non-null  object
 8   Nom_Districte   66423 non-null  object
 9   M_CODI_EIXOS    66423 non-null  object
dtypes: object(10)
memory usage: 5.1+ MB


In [18]:
df_cens[df_cens['BCN3'].str.contains('grup')]

,NOM,Codi_Barri,Nom_Barri,CODI_EIXOS,Codi_Districte,BCN3,BCN2,BCN1,Nom_Districte,M_CODI_EIXOS


In [19]:
# Comprobamos diferencias entre columnas BCN3, BCN2 y BCN1
df_cens[(df_cens['NOM']=='Local Buit') & (df_cens['BCN2']!= 'LOCAL BUIT')]

,NOM,Codi_Barri,Nom_Barri,CODI_EIXOS,Codi_Districte,BCN3,BCN2,BCN1,Nom_Districte,M_CODI_EIXOS
2912,Local Buit,27,el Putxet i el Farró,moda,05,VESTIR,EQUIPAMENT PERSONAL,COMERÇ AL DETALL,Sarrià-Sant Gervasi,pers
3833,Local Buit,19,les Corts,ofic,04,ALTRES,SERVEIS,SERVEIS,Les Corts,serv
9869,Local Buit,25,Sant Gervasi - la Bonanova,moda,05,VESTIR,EQUIPAMENT PERSONAL,COMERÇ AL DETALL,Sarrià-Sant Gervasi,pers
11022,Local Buit,26,Sant Gervasi - Galvany,moda,05,VESTIR,EQUIPAMENT PERSONAL,COMERÇ AL DETALL,Sarrià-Sant Gervasi,pers
12407,Local Buit,17,Sants - Badal,colma,03,ALIMENTACIÓ,QUOTIDIÀ ALIMENTARI,COMERÇ AL DETALL,Sants-Montjuïc,quot
12413,Local Buit,46,el Turó de la Peira,locu,08,ALTRES,SERVEIS,SERVEIS,Nou Barris,serv
15234,Local Buit,60,Sant Andreu,tall,09,REPARACIONS (ELECTRODOMÈSTICS I AUTOMÒBILS),SERVEIS,SERVEIS,Sant Andreu,tran
16017,Local Buit,18,Sants,inmo,03,ACTIVITATS IMMOBILIÀRIES,SERVEIS,SERVEIS,Sants-Montjuïc,serv
18470,Local Buit,44,Vilapicina i la Torre Llobeta,ofic,08,ALTRES,SERVEIS,SERVEIS,Nou Barris,serv
19589,Local Buit,18,Sants,arranj,03,"MANTENIMENT, NETEJA I PRODUCCIÓ",SERVEIS,SERVEIS,Sants-Montjuïc,serv


In [20]:
# Entender mejor los datos que contiene, vemos que se parece a las columnas de BCN3,2 y 1
df_cens['M_CODI_EIXOS'].value_counts()

M_CODI_EIXOS
serv      16895
host      10249
locb       9715
pers       6835
quot       6104
ALTRES     4069
altr       2958
llar       2785
tran       2522
cult       2499
other      1792
Name: count, dtype: int64

In [21]:
df_cens['BCN1'].value_counts()

BCN1
SERVEIS             31239
COMERÇ AL DETALL    21400
LOCAL BUIT           9715
ALTRES               4069
Name: count, dtype: int64

In [22]:
df_cens['BCN3'].value_counts()

BCN3
ALTRES                                                            17914
RESTAURANTS, BARS I HOTELS (INCLÒS HOSTALS, PENSIONS I FONDES)    10249
LOCAL BUIT                                                         9715
VESTIR                                                             3488
ALIMENTACIÓ                                                        2406
ENSENYAMENT                                                        2239
PA, PASTISSERIA I LÀCTICS                                          1705
MOBLES I ARTICLES FUSTA I METALL                                   1639
FINANCES I ASSEGURANCES                                            1551
SANITAT I ASSISTÈNCIA                                              1385
REPARACIONS (ELECTRODOMÈSTICS I AUTOMÒBILS)                        1283
LLIBRES, DIARIS I REVISTES                                         1125
FARMÀCIES                                                          1065
FRUITES I VERDURES                                         

In [23]:
df_cens['BCN2'].value_counts()

BCN2
SERVEIS                   30780
LOCAL BUIT                 9715
QUOTIDIÀ ALIMENTARI        6569
ALTRES                     6277
EQUIPAMENT PERSONAL        5114
PARAMENT DE LA LLAR        3086
OCI I CULTURA              2798
QUOTIDIÀ NO ALIMENTARI     1724
AUTOMOCIÓ                   360
Name: count, dtype: int64

In [24]:
df_cens.columns

Index(['NOM', 'Codi_Barri', 'Nom_Barri', 'CODI_EIXOS', 'Codi_Districte',
       'BCN3', 'BCN2', 'BCN1', 'Nom_Districte', 'M_CODI_EIXOS'],
      dtype='object')

In [25]:
# Eliminamos más columnas que vimos que contiene valores parecidos para reducir nuestro dataset de trabajo
df_cens.drop(['NOM','CODI_EIXOS','BCN3','BCN1','M_CODI_EIXOS'],axis=1,inplace=True)

In [26]:
df_cens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66423 entries, 0 to 66422
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Codi_Barri      66423 non-null  object
 1   Nom_Barri       66423 non-null  object
 2   Codi_Districte  66423 non-null  object
 3   BCN2            66423 non-null  object
 4   Nom_Districte   66423 non-null  object
dtypes: object(5)
memory usage: 2.5+ MB


In [27]:
# Cambio nombre columna por un nombre más adecuado al contenido
df_cens.rename(columns = {'BCN2':'Activitat_Economica'},inplace=True)

In [28]:
df_cens.head()

,Codi_Barri,Nom_Barri,Codi_Districte,Activitat_Economica,Nom_Districte
0,69,Diagonal Mar i el Front Marítim del Poblenou,10,ALTRES,Sant Martí
1,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí
2,69,Diagonal Mar i el Front Marítim del Poblenou,10,SERVEIS,Sant Martí
3,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí
4,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí


In [29]:
report_df(df_cens)

El tamaño del dataset es 66423 filas y 5 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66423 entries, 0 to 66422
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Codi_Barri           66423 non-null  object
 1   Nom_Barri            66423 non-null  object
 2   Codi_Districte       66423 non-null  object
 3   Activitat_Economica  66423 non-null  object
 4   Nom_Districte        66423 non-null  object
dtypes: object(5)
memory usage: 2.5+ MB
None

Tenemos un total de 0 nulos

Hay 65818 duplicados


In [30]:
# Groupby para calcular el total de actividades que hay según el barrio y el tipo de actividad economica
Activitat_Barris = df_cens.groupby(['Codi_Barri','Activitat_Economica']).size().to_frame(name = 'Total_Activitat_Per_Barri').reset_index()

In [31]:
# Imprimimos el dataframe generado para ver como queda y ver si tiene  sentido
Activitat_Barris

,Codi_Barri,Activitat_Economica,Total_Activitat_Per_Barri
0,01,ALTRES,381
1,01,AUTOMOCIÓ,1
2,01,EQUIPAMENT PERSONAL,225
3,01,LOCAL BUIT,245
4,01,OCI I CULTURA,88
...,...,...,...
600,73,OCI I CULTURA,26
601,73,PARAMENT DE LA LLAR,43
602,73,QUOTIDIÀ ALIMENTARI,116
603,73,QUOTIDIÀ NO ALIMENTARI,28


In [32]:
# Comprobación visual de un barrio aleatorio
Activitat_Barris[Activitat_Barris['Codi_Barri']=='69']

,Codi_Barri,Activitat_Economica,Total_Activitat_Per_Barri
561,69,ALTRES,20
562,69,EQUIPAMENT PERSONAL,8
563,69,LOCAL BUIT,42
564,69,OCI I CULTURA,17
565,69,PARAMENT DE LA LLAR,3
566,69,QUOTIDIÀ ALIMENTARI,19
567,69,QUOTIDIÀ NO ALIMENTARI,11
568,69,SERVEIS,185


In [33]:
# merge del dataframe original con el dataframe generado con el groupby anterior. Comprobamos que no se haga un cartesiano
print(df_cens.shape)

df_cens = pd.merge(df_cens,Activitat_Barris, on = ['Codi_Barri','Activitat_Economica'], how = 'left')

print(df_cens.shape)

(66423, 5)
(66423, 6)


In [34]:
# imprimimos por pantalla para comprobar como queda
df_cens.head()

,Codi_Barri,Nom_Barri,Codi_Districte,Activitat_Economica,Nom_Districte,Total_Activitat_Per_Barri
0,69,Diagonal Mar i el Front Marítim del Poblenou,10,ALTRES,Sant Martí,20
1,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí,42
2,69,Diagonal Mar i el Front Marítim del Poblenou,10,SERVEIS,Sant Martí,185
3,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí,42
4,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí,42


In [35]:
# pivot table para que las actividades economicas se coniertan en columnas y reducir un poco la dimensionalidad
df_cens_pivot = pd.pivot_table(df_cens, values='Total_Activitat_Per_Barri', index=['Codi_Districte','Nom_Districte','Codi_Barri','Nom_Barri'], columns='Activitat_Economica', fill_value=0)

In [36]:
df_cens_pivot.reset_index(inplace=True)

In [37]:
# cambio de nombre de columnas para estar alineados con el equipo de Back
df_cens_pivot.rename(columns = {
    'Codi_Districte':'codiDistricte',
    'Nom_Districte' : 'nomDistricte',
    'Nom_Barri':'nomBarri',
    'Codi_Barri':'codiBarri',
    'Nom_Barri':'nomBarri',
    'ALTRES':'nAltres',
    'AUTOMOCIÓ':'nAutomocio',
    'EQUIPAMENT PERSONAL':'nEquipamentPersonal',
    'LOCAL BUIT' : 'nLocalBuit',
    'OCI I CULTURA': 'nOciCultura',
    'PARAMENT DE LA LLAR':'nParamentLlar',
    'QUOTIDIÀ ALIMENTARI' : 'nQuotidiaAlimentari',
    'QUOTIDIÀ NO ALIMENTARI' : 'nQuotidiaNoAlimentari',
    'SERVEIS':'nServeis'
    
},inplace=True

)

In [38]:
# generamos una nueva columna con el año del dataset para más tarde añadir otro años
df_cens_pivot['Anio'] = '2014'

In [39]:
df_cens_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   codiDistricte          73 non-null     object
 1   nomDistricte           73 non-null     object
 2   codiBarri              73 non-null     object
 3   nomBarri               73 non-null     object
 4   nAltres                73 non-null     int64 
 5   nAutomocio             73 non-null     int64 
 6   nEquipamentPersonal    73 non-null     int64 
 7   nLocalBuit             73 non-null     int64 
 8   nOciCultura            73 non-null     int64 
 9   nParamentLlar          73 non-null     int64 
 10  nQuotidiaAlimentari    73 non-null     int64 
 11  nQuotidiaNoAlimentari  73 non-null     int64 
 12  nServeis               73 non-null     int64 
 13  Anio                   73 non-null     object
dtypes: int64(9), object(5)
memory usage: 8.1+ KB


In [40]:
# Convertimos el año en formato fecha
df_cens_pivot['Anio'] = pd.to_datetime(df_cens_pivot['Anio'],format = "%Y")

In [41]:
df_cens_pivot

Activitat_Economica,codiDistricte,nomDistricte,codiBarri,nomBarri,nAltres,nAutomocio,nEquipamentPersonal,nLocalBuit,nOciCultura,nParamentLlar,nQuotidiaAlimentari,nQuotidiaNoAlimentari,nServeis,Anio
0,01,Ciutat Vella,01,el Raval,381,1,225,245,88,137,296,47,1120,2014-01-01
1,01,Ciutat Vella,02,el Barri Gòtic,315,0,478,199,78,76,161,37,786,2014-01-01
2,01,Ciutat Vella,03,la Barceloneta,99,0,26,108,24,13,89,16,370,2014-01-01
3,01,Ciutat Vella,04,"Sant Pere, Santa Caterina i la Ribera",259,1,363,155,53,98,175,34,688,2014-01-01
4,02,Eixample,05,el Fort Pienc,81,12,40,199,47,55,113,33,556,2014-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,20,0,8,42,17,3,19,11,185,2014-01-01
69,10,Sant Martí,70,el Besòs i el Maresme,90,1,16,59,12,22,71,18,204,2014-01-01
70,10,Sant Martí,71,Provençals del Poblenou,166,5,11,122,26,16,40,19,314,2014-01-01
71,10,Sant Martí,72,Sant Martí de Provençals,80,4,40,120,23,34,84,18,336,2014-01-01


# TO JSON

In [42]:
# convertimos nuestro dataset a formato json para pasar al equipo de Back
df_censo_json = df_cens_pivot.to_json(orient='records', lines=True)

In [45]:
file_path = "..\outputs\df_censo_json.json"

with open(file_path, 'w') as file:
    file.write(df_censo_json)